# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint



### Not for Grading

### AI/ML Technique

Automatic colorization functionality for Real-Time User-Guided Image Colorization with Learned Deep Priors, SIGGRAPH 2017!

**Abstract:** A deep learning approach for user-guided image colorization. The system directly maps a grayscale image, along with sparse, local user ``hints" to an output colorization with a Convolutional Neural Network (CNN). Rather than using hand-defined rules, the network propagates user edits by fusing low-level cues along with high-level semantic information, learned from large-scale data. We train on a million images, with simulated user inputs. To guide the user towards efficient input selection, the system recommends likely colors based on the input image and current user inputs. The colorization is performed in a single feed-forward pass, enabling real-time use. Even with randomly simulated user inputs, we show that the proposed system helps novice users quickly create realistic colorizations, and show large improvements in colorization quality with just a minute of use. In addition, we show that the framework can incorporate other user "hints" as to the desired colorization, showing an application to color histogram transfer.

### Setup Steps

In [ ]:
#@title Please enter your registration id to start:  { run: "auto", display-mode: "form" }
Id = "P21A17E_test" #@param {type:"string"}

In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "9812345678" #@param {type:"string"}


In [ ]:
! wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/colorization.zip
! unzip colorization.zip

In [ ]:
cd /content/colorization

#### Importing required packages

In [ ]:
import matplotlib.pyplot as plt

from base_color import *
from eccv16 import *
from siggraph17 import *

from PIL import Image
import numpy as np
from skimage import color
import torch
import torch.nn.functional as F

#### Load the image and pre-process

**load_img()** function is used to load the image using the path of the image

**resize_img()** function is used to resize the image to 256, 256

**preprocess_img()**  function is to convert the image from the sRGB color space (IEC 61966-2-1:1999) to the CIE Lab colorspace under the given illuminant and observer return the image in tensor array.

**postprocess_tens()** function will take the output of the model and it will make the image suitable to plot using matplotlib

In [ ]:
def load_img(img_path):
	out_np = np.asarray(Image.open(img_path))
	if(out_np.ndim==2):
		out_np = np.tile(out_np[:,:,None],3)
	return out_np

def resize_img(img, HW=(256,256), resample=3):
	return np.asarray(Image.fromarray(img).resize((HW[1],HW[0]), resample=resample))

def preprocess_img(img_rgb_orig, HW=(256,256), resample=3):
	# return original size L and resized L as torch Tensors
	img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)
	
	img_lab_orig = color.rgb2lab(img_rgb_orig)
	img_lab_rs = color.rgb2lab(img_rgb_rs)

	img_l_orig = img_lab_orig[:,:,0]
	img_l_rs = img_lab_rs[:,:,0]

	tens_orig_l = torch.Tensor(img_l_orig)[None,None,:,:]
	tens_rs_l = torch.Tensor(img_l_rs)[None,None,:,:]

	return (tens_orig_l, tens_rs_l)

def postprocess_tens(tens_orig_l, out_ab, mode='bilinear'):
	# tens_orig_l 	1 x 1 x H_orig x W_orig
	# out_ab 		1 x 2 x H x W

	HW_orig = tens_orig_l.shape[2:]
	HW = out_ab.shape[2:]

	# call resize function if needed
	if HW_orig[0] != HW[0] or HW_orig[1]!=HW[1]:
		out_ab_orig = F.interpolate(out_ab, size=HW_orig, mode='bilinear')
	else:
		out_ab_orig = out_ab

	out_lab_orig = torch.cat((tens_orig_l, out_ab_orig), dim=1)
	return color.lab2rgb(out_lab_orig.data.cpu().numpy()[0,...].transpose((1,2,0)))

#### Load the pre-trained models convert to cuda runtime

In [ ]:
use_gpu = True

# load colorizers
colorizer_eccv16 = eccv16(pretrained=True).eval()
colorizer_siggraph17 = siggraph17(pretrained=True).eval()
if use_gpu:
	colorizer_eccv16.cuda()
	colorizer_siggraph17.cuda()

specify the image path to pre-process and convert the image tensor array to cuda runtime

In [ ]:
img_path = 'BlacknWhite.JPG'

# default size to process images is 256x256
# grab L channel in both original ("orig") and resized ("rs") resolutions
img = load_img(img_path)
(tens_l_orig, tens_l_rs) = preprocess_img(img, HW=(256,256))
if(use_gpu):
	tens_l_rs = tens_l_rs.cuda()

#### Model evaluation and save the output

In [ ]:
# colorizer outputs 256x256 ab map
# resize and concatenate to original L channel
img_bw = postprocess_tens(tens_l_orig, torch.cat((0*tens_l_orig,0*tens_l_orig),dim=1))
out_img_eccv16 = postprocess_tens(tens_l_orig, colorizer_eccv16(tens_l_rs).cpu())
out_img_siggraph17 = postprocess_tens(tens_l_orig, colorizer_siggraph17(tens_l_rs).cpu())

plt.imsave('s_eccv16.png', out_img_eccv16)
plt.imsave('s_siggraph17.png', out_img_siggraph17)

#### Plot the image

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
plt.imshow(img)
plt.title('Original')
plt.axis('off')

plt.subplot(2,2,2)
plt.imshow(img_bw)
plt.title('Input')
plt.axis('off')

plt.subplot(2,2,3)
plt.imshow(out_img_eccv16)
plt.title('Output (ECCV 16)')
plt.axis('off')

plt.subplot(2,2,4)
plt.imshow(out_img_siggraph17)
plt.title('Output (SIGGRAPH 17)')
plt.axis('off')
plt.show()

References: https://arxiv.org/abs/1603.08511